In [ ]:
# # Using modin for faster dataframe processing
# import ray
# ray.init()
# import modin.pandas as pd
import pandas as pd

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

import re
from dateutil import parser as date_parser
from datetime import datetime
import geocoder

from scipy import stats
import numpy as np

import spacy
# spacy.cli.download('en_core_web_lg')

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

# config contains config required for bigquery table
from config import table_id
from google.cloud import bigquery

Read data into dataframe

In [ ]:
df = pd.read_json("ufoReports.json", orient="records", dtype=object, convert_dates=False)
df.info()

Dropping redundant columns

In [ ]:
df = df.drop(columns=['location', 'LOCATION', 'OCCURRED', 'REPORTED', 'DURATION', 'SHAPE'], axis=1)
# Not going to use this data
df = df.drop(columns=['Posted'], axis=1)
df = df.rename(columns={"Characteristics": "Report"})
df.info()

Remove duplicate rows

In [ ]:
duplicate_index = df[df.duplicated(["Occurred", "Location", "Report"])].index
print(f'number of rows={len(df)}, duplicated rows={len(duplicate_index)}')
df = df.drop(duplicate_index)
print(f'number of rows after deletion: {len(df)}')

Covert Occurred and reported date time to proper format

In [ ]:
# used "(O|R|P).*":\s"\s*\d{4} to check date formate in data
def convert_date_time(text):
    try:
        # pattern ='date' 'time' 'AM' or 'PM' if it is mentioned
        date_time_pattern = re.compile(r'\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{1,2}(:\d{1,2})?\s?(AM|PM)?', re.IGNORECASE)
        # timeZone = tz.gettz(TimezoneFinder().timezone_at(latlang))
        string_time = date_time_pattern.search(text).group()
        return date_parser.parse(string_time) if string_time else None
    except Exception:
        return None


df['TimeOfEvent'] = df.Occurred.apply(convert_date_time, convert_dtype=False)
df['ReportedTime'] = df.Reported.apply(convert_date_time, convert_dtype=False)
df.info()

Transform Duration Strings to seconds(float)

In [ ]:
def calculate_seconds(string):
    try:
        # Following commented method fail to parse strings reported in different formate,
        # gosh! have to do string manipulation again
        # dt = date.today()
        # #make dt as midnight time
        # dt = datetime.combine(dt, datetime.min.time())
        # #time from string dn calculated as today's date + time
        # dn = date_parser.parse(string)
        # time_string = str(dn - dt)
        # h, m, s = map(int, time_string.split(':'))
        # return int(h * 3600 + m * 60 + s)

        # extract total seconds using string manipulation
        # TODO: There are some cases which can be parsed with use of word2number package for e.g "one-two minutes"
        # TODO: Unable to parse cases like 2-minutes, figure out different cases for better results
        h_patrn = re.compile(r'(\d{1,2})\s?(hr|hour)', re.IGNORECASE)
        m_patrn = re.compile(r'(\d{1,2})\s?(min|minute)', re.IGNORECASE)
        s_patrn = re.compile(r'(\d{1,2})\s?(sec|second)', re.IGNORECASE)
        try:
            h = int(h_patrn.search(string).group(1))
        except:
            h = 0
        try:
            m = int(m_patrn.search(string).group(1))
        except:
            m = 0
        try:
            s = int(s_patrn.search(string).group(1))
        except:
            s = 0
        total_sec = h * 3600 + m * 60 + s
        # when fail to parse
        if total_sec == 0:
            return None
        else:
            return total_sec
    except Exception:
        return None


df['DurationInSecond'] = df.Duration.apply(calculate_seconds, convert_dtype=False)
df['DurationInSecond'].info()

Clean 'Shape' column. Transform column to have categorical data(one word representing shape).

In [ ]:
# Fill in missing shape value as Unknown and use Unknown for Other type
def fill_shape(x):
    x = x.strip().lower()
    if x:
        if x == "other":
            return "unknown"
        else:
            return x
    else:
        return "unknown"


df['Shape'] = df['Shape'].apply(fill_shape)

# Check if shape column needs further cleaning
shape_uniq = df['Shape'].unique()
print(f"Unique shapes:\n{shape_uniq}")

# make shape category if shape has one word to describe it
cat_filter = []
for shape in shape_uniq:
    # no space = one word since the shapes were stripped earlier
    cat_filter.append(not shape.__contains__(" "))
shape_categories = shape_uniq[cat_filter]
print(f"\nShape categories:\n{shape_categories}")

# Pattern to identify shape from given description
shapes_cat_pattern = re.compile(r'\b(' + r'|'.join(shape_categories) + r')\b')


# try to find shape when sentences are passed instead of one word
def categorize_shape(shape_str):
    # since all shape categories are of one word
    if shape_str.__contains__(" "):
        try:
            # Problematic in cases like: sentence is something like "light coming out of triangle"
            #  then light will be used instead triangle
            #  OR if word triangular is used instead triangle
            shape = shapes_cat_pattern.search(shape_str).group(1)
            return shape
        except:
            return "unknown"
    else:
        return shape_str


# Clean Shape
df['Shape'] = df['Shape'].apply(categorize_shape)
print(df['Shape'].value_counts())
df['Shape'].info()

Remove rows which couldn't be transformed

In [ ]:
rbt = df.shape
# Dropping rows with missing value
df = df.dropna(subset=["TimeOfEvent", "ReportedTime", "Shape"], axis=0, how="any")

# Dropping unnecessary columns
df.drop(columns=["Occurred", "Reported", "Duration"], axis=1)
rat = df.shape
print(df.info())
print(f"\n##Dropping some missing values\n"
      f"rows dropped={rbt[0] - rat[0]}, Before:{rbt}, after:{rat}")

Calculate Day of the event occurred

In [ ]:
df['DayOfEvent'] = df['TimeOfEvent'].apply(lambda x: x.strftime("%A"))
df.info()

Clean 'DurationInSecond' column

In [ ]:
# apply function so missing values can be treated as NaN
df['DurationInSecond'] = df['DurationInSecond'].apply(pd.to_numeric)

# looking for outliers
sns.boxplot(x=df['DurationInSecond'])
plt.show()

In [ ]:
# Scatter plt for duration to check for outliers
sns.scatterplot(x=df['TimeOfEvent'], y=df['DurationInSecond'])
plt.show()

Will Use Z score for outlier removal

In [ ]:
sb = df.shape

# calculate z score to find outliers in DurationInSecond
z = np.abs(stats.zscore(df["DurationInSecond"], nan_policy='omit'))
# as common practice, remove z where abs(z) > 3
delete_indexes = z[z > 3].index
df = df.drop(delete_indexes, axis=0)
sa = df.shape
print(f"\n\nDeleted Outliers where absolute z score of DurationInSecond> 3\n"
      f"rows deleted={sb[0] - sa[0]}")

In [ ]:
# Curious to see if shapes correlate to Duration UFO were visible
sns.scatterplot(x=df['Shape'], y=df['DurationInSecond'])
plt.xticks(rotation=45)
plt.show()

Fill missing Duration in second value

In [ ]:
# replace null duration with mean
# would replace with closest neighbour would have been better instead?
meanDuration = df['DurationInSecond'].mean()
df['DurationInSecond'] = df['DurationInSecond'].fillna(meanDuration)

# Dropping original Duration column since we have transformed it to values in seconds
del (df['Duration'])
df.info()

Some stuff to explore Corpus, and to determine things to consider before finding TFIDF

In [ ]:
# Goal is to give weight to reports based on uniqueness of the report. (We want more Information gain haha)
# Summation of IDF values of tokens in a document can be used to assign the weight
# but using TFIDF can leverage unique repeating words within document.
# However, TFIDF will be more prone to number of tokens in a document compared to only IDF
# since using TF is dependent of doc's length.
# More experiments should be conducted to determine better method to assign weight to the report.


# stopword pattern
stopword_pattern = re.compile(r'(\b(' + r'|'.join(stopwords) + r')(\b))|(\\+n)')
# resetting index to make sure we have unique indexes
df = df.reset_index(drop=True)


# Remove stop words and tokenize report into words with lowercase
def get_useful_tokens(report):
    if not isinstance(report, str):
        return []
    report = report.lower()
    report = stopword_pattern.sub('', report)
    # only using words that starts with alphabets. When tokenizing based on \w only, it extracted  many unnecessary
    # tokens
    token_list = re.findall(r'(\b[a-z]\w+\b)', report)
    return token_list


# Find corpus specific stopwords
def find_c_specific_stopwords():
    # Tokens in corpus with general stopwords removed
    c_tokens = df['Report'].apply(get_useful_tokens)

    # iterate over rows and then withing that iteration iterate over tokens to flatten it
    # indexes are same for tokens in a row
    f_tokens = pd.DataFrame(
        [(index, value) for (index, values) in c_tokens.iteritems()
         for value in values],
        columns=['index', 'tokens']
    ).set_index('index')
    token_count = f_tokens.value_counts()

    # making a representable dataframe
    token_count = token_count.reset_index()
    token_count = token_count.rename(columns={0: "Count"})
    print("Tokens and their counts:\n", token_count)

    # So there are many unnecessary tokens that starts with numbers and there are tokens that are misspelled. Not
    # dealing with misspelled tokens right now because it is time consuming with Spacy's contextualSpellCheck module.
    # If we want to correct spellings, have to find a better way.
    # token_count.to_csv('tokenCounts.csv')

    # using boxplot to check what counts are too many (outliers)
    sns.boxplot(x=token_count['Count'])
    plt.title('What counts are too much compared to others')
    plt.show()

    # if count > 45k calling it a corpus specific stopword
    cond = token_count['Count'] > 45000
    c_stopwords = token_count['tokens'][cond].to_list()
    print("Corpus specific stopwords: ", c_stopwords)
    stopwords.update(c_stopwords)
    return c_stopwords


# Update stopwords with corpus specific stop words
stopwords.update(find_c_specific_stopwords())
stopword_pattern = re.compile(r'(\b(' + r'|'.join(stopwords) + r')(\b))|(\\+n)')

Perform a rule based lemmatization on a given Report (Description by user) then remove stopwords and tokenize words to find TF(term frequency)

In [ ]:
'''
It takes about an hour when using single core. have tried multiple methods but spacy's pipline for lemmatization is our bottleneck
Tried passing docs to cython code for faster iteration over tokens
Tried passing giant text (concatenation of df['Report'] using pandas.series.str.cat)
into pipeline but there is no improvement in timing.
Tried chunking the giant text but the there is no significant improvement since nlp() or spacy is our bottleneck.
There is no significant improvement when pipeline trained over small dataset(en_core_web_sm) is used
so using en_core_web_lg.
'''

nlp = spacy.load("en_core_web_lg", exclude=['parser', 'ner'])
nlp.enable_pipe('senter')
null_doc_count = 0


# Lemmatize Report, convert text to lowercase, and remove stop words
def text_preprocess(report):
    if not isinstance(report, str):
        global null_doc_count
        null_doc_count = null_doc_count + 1
        return ""
    doc = nlp(report)
    lemma_text = ""
    for sent in doc.sents:
        lemma_text = f'{lemma_text}{sent.lemma_}'
    lemma_text = lemma_text.lower()
    return stopword_pattern.sub('', lemma_text)


st = datetime.now()
df['Summary'] = df['Report'].apply(text_preprocess)
print("Time to prep corpus", datetime.now() - st)
df.to_json("computedDF.json")

Calculate score of report by summing up TFID values of tokens in a report

In [ ]:
# Not using TFIDF from sklearn because their method seems to be more prone to number of tokens than regular one
# Check experiment_playground for more detail
# If we want find sum of TFID directly from scipy implementation:
# vectorizer = TfidfVectorizer(lowercase=False, token_pattern=r'(?u)\b[a-z]\w+\b')
# tf_idf_vectors = vectorizer.fit_transform(df['Summary'].to_list())
# return TF.sum(axis=1).A1

# Count vector to count number of tokens in a report
# consider only tokens/words that start with alphabets. It will get rid of some unnecessary tokens
vectorizer = CountVectorizer(lowercase=False, token_pattern=r'(?u)\b[a-z]\w+\b')
TF = vectorizer.fit_transform(df['Summary'].to_list())

# the resultant array will have tokens as column and their count in reports as a row
# token_col_list = vectorizer.get_feature_names_out()

# total number of documents that are not empty
N = df['Summary'].shape[0] - null_doc_count


# Function to calculate IDF
def cal_idf(doc_freq):
    return np.log(N / doc_freq)


# number of tokens in a doc
nt_in_doc = csr_matrix(TF.sum(axis=1).A1).transpose()

# Making a IDF calculating numpy func so we can apply it to array
idf_v = np.vectorize(cal_idf)
# non zero entries across token column gets document frequency
IDF = csr_matrix(idf_v(TF.getnnz(axis=0))).transpose()

# multiply count vector matrix to matrix of token's
# IDF (matrix of one column, rows=IDF of tokens which are in same sequence as token columns in countvector matrix)
TFIDF_SUM = TF._mul_sparse_matrix(IDF)

# Dividing by number of tokens to use general formula of TF
# do division where condition is met otherwise return 0. This will avoid division by 0 problem
score = pd.Series(TFIDF_SUM._divide_sparse(nt_in_doc).A1)
score = score.fillna(0)
df['Report_score'] = score

Geocode Location.

In [ ]:
# It takes around 30 hrs because arcgis api limits 1 api call per second (brutal for our 133k+ records).
# TODO: There might be better approach like api with no limit or with any offline database
# using latitude longitude so it is easy to put on a map and find timeZone if needed
def getlatlang(loc):
    try:
        location = geocoder.arcgis(loc)
        return location.latlng
    except Exception as e:
        print(e)
        return None


df["LatLng"] = df.Location.apply(lambda x: getlatlang(x) if x else None)
df = df.dropna(subset=["LatLng"], axis=0, how="any")

Dropping unnecessary columns

In [ ]:
df = df.drop(columns=['Occurred', 'Reported', 'Summary'], axis=1)
df.info()

For some reason, cant pass lat lng as array or dict or named tuple for bigquery.enums.SqlTypeNames.GEOGRAPHY type

In [ ]:
# TODO: Try using pyproj or pyCRS for Open Geospatial Consortium standard
# We can convert lat long to Geopoint which is supported by GEOGRAPHY data type of bigquery with following
# ALTER TABLE <TableName> ADD COLUMN GeoPoint GEOGRAPHY;
# UPDATE <TableName> SET GeoPoint=ST_GEOGPOINT(Lng, Lat) where TRUE;

df['Lat'] = df['LatLng'].apply(lambda x: x[0])
df['Lng'] = df['LatLng'].apply(lambda x: x[1])
del (df['LatLng'])
df.info()
df.to_csv("~/Desktop/CleanUFOs.csv")

Send this to oblivion (to any external database for safe keeping)

In [ ]:
client = bigquery.Client()
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField('ID', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField('TimeOfEvent', bigquery.enums.SqlTypeNames.DATETIME),
        bigquery.SchemaField('ReportedTime', bigquery.enums.SqlTypeNames.DATETIME),
        bigquery.SchemaField('Location', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField('Lat', bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField('Lng', bigquery.enums.SqlTypeNames.FLOAT64),
        bigquery.SchemaField('Shape', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField('DurationInSecond', bigquery.enums.SqlTypeNames.FLOAT),
        bigquery.SchemaField('Report', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField('Report_score', bigquery.enums.SqlTypeNames.FLOAT64)
    ],
    # Overwrite existing data
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)